<img src="https://raw.githubusercontent.com/PriyeshGosai/pypsa-meets-earth-lab-2025/main/img/top-banner.png" alt="Top Banner" width="100%">

<div style="display: flex; align-items: center; justify-content: space-between; gap: 30px;">
  <div>
    <h3 style="margin-top: 0;">Session Instructor</h3>
    <p style="margin: 5px 0; font-size: 20px;"><strong>Priyesh Gosai</strong></p>
    <p style="margin: 2px 0; font-size: 14px;">PAS-SA • Director: Energy Systems</p>
    <hr style="margin: 12px 0; border: none; border-top: 1px solid #000000ff;">
    <p style="margin: 6px 0; font-size: 13px;"><a href="mailto:pgosai@pas-sa.co.za">pgosai@pas-sa.co.za</a></p>
    <p style="margin: 6px 0; font-size: 13px;"><a href="https://www.linkedin.com/in/gosaip/">LinkedIn</a> | <a href="https://github.com/PriyeshGosai">GitHub</a> | <a href="https://www.pas-sa.co.za">Website</a></p>
  </div>
  <div>
    <a href="https://pas-sa.co.za/">
      <img src="https://raw.githubusercontent.com/PriyeshGosai/pypsa-meets-earth-lab-2025/main/img/pas-sa-logo.png" width="160" alt="PAS-SA">
    </a>
  </div>
</div>

# **Part 1: Getting started with PyPSA**



## Prepare Google Colab Environment

In [ ]:
import os

#@title Install Packages {display-mode:"form"}
INSTALL_PACKAGES = True #@param {type:"boolean"}

# Check if packages have already been installed in this session to prevent re-installation
if INSTALL_PACKAGES and not os.environ.get('PYPSA_PACKAGES_INSTALLED'):
  !pip install pypsa pypsa[excel] folium mapclassify cartopy
  os.environ['PYPSA_PACKAGES_INSTALLED'] = 'true'
elif not INSTALL_PACKAGES:
  print("Skipping package installation.")
else:
  print("PyPSA packages are already installed for this session.")

# Exercise 1:

**Objective:**

Programatically build a PyPSA network, define the model’s constraints, solve it, and review the results.

**Diagram**

<img src="https://raw.githubusercontent.com/PriyeshGosai/pypsa-meets-earth-lab-2025/main/img/example_1_img.png" alt="Example 1" width="50%">

**Steps**
1. Create the network object

In [ ]:
import pypsa
import pandas as pd
pypsa.options.api.new_components_api = True

n = pypsa.Network()

n.add('Carrier','gas')
n.add('Carrier','AC')
n.add('Bus','Location',carrier = 'AC')
n.add('Load','Load A',bus = 'Location',p_set = 100,carrier = 'gas')
n.add('Generator','Generator A',bus = 'Location', p_nom = 500, marginal_cost = 1,p_nom_extendable = True)
n.set_snapshots(pd.date_range('2025-01-01', '2025-12-31 23:00', freq='h'))

n.optimize()


2. Use the add method to add components.
3. Prepare the snapshots for a single investment period with an hourly resolution. 

In [ ]:
n.generators.static.p_nom_opt

In [ ]:
n.objective

In [ ]:
n.objective_constant

4. Observe components stored as Pandas dataframes

In [ ]:
n.generators.static

In [ ]:
n.loads.static

In [ ]:
n.buses.static

5. Build the full optimisation model (variables, constraints, objective).

In [ ]:
n.optimize.create_model()

6. Pass that model to the solver and gets the results.

In [ ]:
n.optimize.solve_model()


We can also run both steps in sequence.

Therefore: 

```

n.optimize()

```

is the same as:


```

n.create_model()

n.solve_model()

```








7. Observe the results using the methods from pandas. 

* `plot()`
* `describe()`
* `sum()`

Since we are using the new api, we need to call is as follows:

`n.generators.dynamic.p.plot()`

In [ ]:
n.generators.static

In [ ]:
n.objective_constant

In [ ]:
n.buses.dynamic.marginal_price.sum()

In [ ]:
n.snapshot_weightings

In [ ]:
n.objective

In [ ]:
for key in n.buses.dynamic:
    print(key)

In [ ]:
n.loads

In [ ]:
n.objective

In [ ]:
print('='*60)
print('Generator Results')
print('='*60)
print(f'The total power generated over the period for all generators: {n.generators.dynamic.p.sum().sum()}')
print('describe()')
print(n.generators.dynamic.p.describe())
n.generators.dynamic.p.plot()

In [ ]:
print('='*60)
print('Bus Results')
print('='*60)

n.generators.dynamic.p.plot()

# Example 2:

In this example, we will use an example network distributed with PyPSA to observe more complex features. 



This example demonstrates how to optimise meshed AC-DC networks in PyPSA. The example has a 3-node AC network coupled via AC-DC converters to a 3-node DC network. There is also a single point-to-point DC connection using the Link component.


[Meshed AC-DC Network Example](https://docs.pypsa.org/latest/examples/ac-dc-lopf/)

In [ ]:
import pypsa
pypsa.options.api.new_components_api = True

network = pypsa.examples.ac_dc_meshed()

In [ ]:
network.carriers.static

In [ ]:
network.carriers.static

In [ ]:
network.global_constraints.static

In [ ]:
network.generators.static.efficiency

In [ ]:
network.generators.static.p_nom_extendable

In [ ]:
network.generators.static

View the network on a map

In [ ]:
line_color = network.lines.static.bus0.map(network.buses.static.carrier).map(
    lambda ct: "r" if ct == "DC" else "b"
)

network.plot.explore(
    line_color=line_color,
    link_color="c",
    jitter=0.4,
)

View all the components in the network

In [ ]:
network.determine_network_topology()


In [ ]:
network.snapshots

In [ ]:
network.buses.static

In [ ]:
network.generators.static

Constraints applied to generators based on constraints at time steps.

e.g. Maximum solar/wind or for conventional generators that have restrictions due to cooling systems.

In [ ]:
network.generators.dynamic.p_max_pu

In [ ]:
network.lines.static

In [ ]:
network.links.static

In [ ]:
network.loads.static

Timeseries data for loads

In [ ]:
network.loads.dynamic.p_set

In [ ]:
network.loads.dynamic.p_set.plot()

Global constraints

In [ ]:
network.global_constraints.static

Subnetworks

Sub-networks are built in the PyPSA network and provides a hierarchical way to structure your PyPSA model, making it more robust, interpretable, and flexible. 



* Sub-networks in PyPSA are a powerful feature for organizing and analyzing complex energy systems. 
* They represent distinct, interconnected parts of a larger network, allowing for a more structured and manageable approach to modeling and optimization. 
* Sub-networks allow you to break down the network into smaller, more understandable modules. This makes the model easier to build, understand, and maintain.
* Sub-networks help in understanding the interactions and dependencies between different parts of the system. For example, you might define separate sub-networks for AC and DC grids, or for different geographical regions, and then analyze how power flows between them.
* Different sub-networks can have distinct electrical characteristics (e.g., AC vs. DC, different voltage levels) or operational rules. PyPSA's sub-network feature allows you to capture these differences accurately within a unified framework.
* For certain types of analysis, especially in optimization, dealing with smaller sub-problems can sometimes improve computational efficiency, although PyPSA generally optimizes across the entire network.


In [ ]:
network.sub_networks.static

In [ ]:
network.sub_networks.static.loc['0','obj']

In [ ]:
network.sub_networks.static.loc['0','obj'].components.buses.static

Solve the model

In [ ]:
network.optimize()

View all the constraints

In [ ]:
network.model

View Results

In [ ]:
network.generators_t.p.plot()

In [ ]:
network.links.dynamic.p0.plot()

In [ ]:
network.lines.dynamic.p0.plot()

Export the results file

In [ ]:
file_name_solved_ac_dc = 'solved_ac-dc.xlsx'
network.export_to_excel(file_name_solved_ac_dc)
